In [7]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=4, app_name='pnc', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 4 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: pnc...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:26


In [9]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


PC  
- 06/08 ~ 06/17
  
- daily  
- by membership. 
- unit sold = pu  

- CurrencyIncreased  
    - 투표 획득 경로

In [119]:
from pyspark.sql.types import * 

def classify_country(country_os, country_ip):
    if country_os != 'CN':
        return country_ip
    else:
        return country_os

country_type_udf = udf(classify_country, StringType())

meta_region = mysql.read_table(spark, 'metainfo', 'meta_bi_regions')

user = load_data_mart("pc", "2022-06-17", "2022-06-17", "user_master").where(col('lastlogindate') >= "2022-06-08")
user = user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
user = user.join(meta_region, user.country_new == meta_region.country_code_iso2, "left")

In [120]:
gcoin = load_data_mart("pc", "2022-06-08", "2022-06-17", "gcoin_use").where(col("event_name").like("%pnc%"))
user = user.withColumnRenamed("accountid", "account_id")
membership = load_membership("pubg", "pc", "2022-06-17", "2022-06-17")
user = user.join(membership, "account_id")

In [61]:
pnc = gcoin.join(user, "account_id", "left")

In [62]:
region_au = user.groupBy("pubg_region").agg(countDistinct("account_id").alias("au"))

In [63]:
region_pu = pnc.groupBy("pubg_region").agg(countDistinct("account_id").alias("pu"), sum("free_use").alias("free_use"), sum("paid_use").alias("paid_use"))

In [64]:
region_paid_pu = pnc.where(col("paid_use") > 0).groupBy("pubg_region").agg(countDistinct("account_id").alias("paid_pu"))

In [65]:
region_pu = region_pu.join(region_paid_pu, "pubg_region")

In [66]:
region_kpi = region_au.join(region_pu, "pubg_region").withColumn("pur", col("pu")/col("au")).withColumn("paid_pur", col("paid_pu")/col("au")).withColumn("total_sales", (col("free_use") + col("paid_use"))/100).withColumn("paid_sales", col("paid_use")/100).toPandas()

In [67]:
region_kpi

,pubg_region,au,pu,free_use,paid_use,paid_pu,pur,paid_pur,total_sales,paid_sales
0,CN,3357381,19181,10451930,18127760,15210,0.005713,0.004530,285796.9,181277.6
1,NA,139160,792,381780,798300,661,0.005691,0.004750,11800.8,7983.0
2,SA,128951,910,671580,684320,635,0.007057,0.004924,13559.0,6843.2
3,KR,641019,4859,1926520,5313390,4242,0.007580,0.006618,72399.1,53133.9
4,SEA,237245,1824,1108450,1609310,1447,0.007688,0.006099,27177.6,16093.1
5,JP,17820,214,117370,201490,173,0.012009,0.009708,3188.6,2014.9
6,TW/HK,44664,408,218280,389640,335,0.009135,0.007500,6079.2,3896.4
7,CIS,382258,1192,881530,894550,798,0.003118,0.002088,17760.8,8945.5
8,EMEA,436769,2578,1499150,2342070,1985,0.005902,0.004545,38412.2,23420.7
9,OC,22374,196,89270,202770,175,0.008760,0.007822,2920.4,2027.7


In [70]:
membership_au = user.groupBy("membership").agg(countDistinct("account_id").alias("au"))
membership_pu = pnc.groupBy("membership").agg(countDistinct("account_id").alias("pu"), sum("free_use").alias("free_use"), sum("paid_use").alias("paid_use"))
membership_paid_pu = pnc.where(col("paid_use") > 0).groupBy("membership").agg(countDistinct("account_id").alias("paid_pu"))
membership_pu = membership_pu.join(membership_paid_pu, "membership")

membership_kpi = membership_au.join(membership_pu, "membership").withColumn("pur", col("pu")/col("au")).withColumn("paid_pur", col("paid_pu")/col("au")).withColumn("total_sales", (col("free_use") + col("paid_use"))/100).withColumn("paid_sales", col("paid_use")/100).toPandas()

In [71]:
membership_kpi

,membership,au,pu,free_use,paid_use,paid_pu,pur,paid_pur,total_sales,paid_sales
0,plus,144323,3378,1888720,3144500,2715,0.023406,0.018812,50332.2,31445.0
1,basic,2157774,993,398450,1081120,873,0.000460,0.000405,14795.7,10811.2
2,legacy,3105901,27783,15058690,26337980,22073,0.008945,0.007107,413966.7,263379.8


- Total G-Coin: 47,909,460 G-Coin  
- Paid G-Coin: 30,563,600 G-Coin

- pu = 32154  
- paid pu = 25661

In [24]:
env = 'live'
log_type = 'PickStatusChanged'

pick_status = load_schema.lobby(spark, "pc", env, log_type, "2022-06-08", "2022-06-17")

In [118]:
env = 'live'
log_type = 'CurrencyIncreased'

currency = load_schema.lobby(spark, "pc", env, log_type, "2022-06-08", "2022-06-17")

In [121]:
currency = currency.withColumnRenamed("AccountId", "account_id")
user_tmp = user.select("account_id", "membership", "pubg_region")
currency_pnc = currency.where((col("Currency").like("%pickem%")) & ((col("Reason") != "inventory-cleanup"))).join(user_tmp, "account_id", "left")

In [101]:
currency_pnc = currency_pnc.where((~col("Currency").isin(["currencydesc.pickem.votingcoupon.p.pickem202102", "currencydesc.pickem.votingcoupon.p.pickem202101"])))

In [114]:
daily_currency = currency_pnc.withColumn("case", when(col("amount") > 0, "+").otherwise("-")).groupBy("date", "case").agg(sum("Amount").alias("amount"), countDistinct("account_id").alias("user_cnt")).orderBy("date", "case").toPandas()

In [115]:
daily_currency

,date,case,amount,user_cnt
0,2022-06-08,+,556391,505991
1,2022-06-08,-,-299876,280634
2,2022-06-09,+,317449,298900
3,2022-06-09,-,-222004,210487
4,2022-06-10,+,218165,204089
5,2022-06-10,-,-152008,142489
6,2022-06-11,+,197544,184741
7,2022-06-11,-,-139728,130864
8,2022-06-12,+,153234,142615
9,2022-06-12,-,-111381,103349


In [122]:
daily_currency_reason = currency_pnc.withColumn("case", when(col("amount") > 0, "+").otherwise("-")).groupBy("date", "case", "Reason").agg(sum("Amount").alias("amount"), countDistinct("account_id").alias("user_cnt")).orderBy("date", "case").toPandas()

In [124]:
daily_currency_reason.head()

,date,case,Reason,amount,user_cnt
0,2022-06-08,+,commercial-word-redeem,5,5
1,2022-06-08,+,outgameevent-mission-reward,502331,502331
2,2022-06-08,+,buy-cash-item,54055,10811
3,2022-06-08,-,pickem-pick,-299876,280634
4,2022-06-09,+,open-crate,1,1


In [125]:
daily_currency_reason.to_csv("./daily_currency_reason.csv", index=False)

In [130]:
currency_pnc.withColumn("case", when(col("amount") > 0, "+").otherwise("-")).groupBy("case").agg(countDistinct("account_id").alias("user_cnt")).show()

+----+--------+
|case|user_cnt|
+----+--------+
|   +| 1654784|
|   -| 1123819|
+----+--------+



In [131]:
voting_coupon_by_user = currency_pnc.withColumn("case", when(col("amount") > 0, "+").otherwise("-")).groupBy("case", "account_id").agg(sum("Amount").alias("amount")).toPandas()

In [132]:
voting_coupon_by_user.groupby("case")["amount"].mean()

case
+    1.111427
-   -1.121165
Name: amount, dtype: float64

In [100]:
pickem_region = currency_pnc.groupBy("pubg_region", "Currency", "Reason").agg(sum("Amount").alias("amount")).toPandas()

In [93]:
pickem_region.groupby(["pubg_region", "Reason"])["amount"].sum().reset_index()

,pubg_region,Reason,amount
0,CIS,buy-cash-item,5960
1,CIS,commercial-word-redeem,4827
2,CIS,open-crate,10
3,CIS,outgameevent-mission-reward,117321
4,CIS,pickem-pick,-76710
5,CIS,promotion-code-redeem,23
6,CN,buy-cash-item,95905
7,CN,commercial-word-redeem,13424
8,CN,open-crate,55
9,CN,outgameevent-mission-reward,1070121


In [133]:
pickem_region.to_csv("./pickem_region.csv", index=False)

In [94]:
pickem_region.groupby("pubg_region")["amount"].sum().reset_index()

,pubg_region,amount
0,CIS,51431
1,CN,354837
2,EMEA,50173
3,JP,1746
4,KR,62993
5,NA,11932
6,OC,2512
7,SA,15184
8,SEA,24122
9,TW/HK,4248


In [95]:
pickem_region[pickem_region.amount > 0].groupby("pubg_region")["amount"].sum().reset_index()

,pubg_region,amount
0,CIS,128141
1,CN,1179575
2,EMEA,131916
3,JP,4943
4,KR,216775
5,NA,28757
6,OC,7066
7,SA,42150
8,SEA,84097
9,TW/HK,15745


In [98]:
pickem_region[pickem_region.amount < 0].groupby("pubg_region")["amount"].sum().reset_index()

,pubg_region,amount
0,CIS,-76710
1,CN,-824738
2,EMEA,-81743
3,JP,-3197
4,KR,-153782
5,NA,-16825
6,OC,-4554
7,SA,-26966
8,SEA,-59975
9,TW/HK,-11497


In [106]:
currency_pnc.withColumn("case", when(col("amount") > 0, "+").otherwise("-")).groupBy("pubg_region", "case").agg(sum("Amount").alias("amount"), countDistinct("account_id").alias("user_cnt")).orderBy("pubg_region", "case").toPandas()

,pubg_region,case,amount,user_cnt
0,CIS,+,128141,117717
1,CIS,-,-76710,68508
2,CN,+,1179575,1074855
3,CN,-,-824738,751190
4,EMEA,+,131916,115698
5,EMEA,-,-81743,69327
6,JP,+,4943,3750
7,JP,-,-3197,2176
8,KR,+,216775,191752
9,KR,-,-153782,134204


In [108]:
currency_pnc.withColumn("case", when(col("amount") > 0, "+").otherwise("-")).groupBy("membership", "case")\
    .agg(sum("Amount").alias("amount"), countDistinct("account_id").alias("user_cnt")).orderBy("membership", "case").toPandas()

,membership,case,amount,user_cnt
0,basic,+,313540,307385
1,basic,-,-169593,165644
2,legacy,+,1417227,1256605
3,legacy,-,-1010951,891406
4,plus,+,108399,90794
5,plus,-,-79443,66769


In [25]:
currency_increased = mysql.read_table(spark, "labs", "hourly_currency_increased").where((col("datetime") >= "2022-06-08") & (col("datetime") <= "2022-06-17"))

In [30]:
currency_increased.where((col("Currency").like("%pickem%")) & (col("Reason") != "inventory-cleanup")).show(truncate=False)

+-------------------+-----------------------------------------------+---------------------------+--------+-------------------+------------------+
|datetime           |Currency                                       |Reason                     |Provider|regdatetime        |currency_increased|
+-------------------+-----------------------------------------------+---------------------------+--------+-------------------+------------------+
|2022-06-08 02:00:00|currencydesc.pickem.votingcoupon.p             |pickem-pick                |steam   |2022-06-08 04:23:05|-1                |
|2022-06-08 03:00:00|currencydesc.pickem.votingcoupon.p.pickem202206|buy-cash-item              |kakao   |2022-06-08 04:23:05|5                 |
|2022-06-08 02:00:00|currencydesc.pickem.votingcoupon.p.pickem202206|buy-cash-item              |steam   |2022-06-08 04:23:05|5                 |
|2022-06-08 05:00:00|currencydesc.pickem.votingcoupon.p             |pickem-pick                |kakao   |2022-06-08 06:37:3

In [31]:
pickem = currency_increased.where((col("Currency").like("%pickem%")) & (col("Reason") != "inventory-cleanup")).groupBy("Currency", "Reason").agg(sum("currency_increased")).toPandas()

In [38]:
pickem.loc[1:8]

,Currency,Reason,sum(currency_increased)
1,currencydesc.pickem.votingcoupon.p,pickem-pick,-116307
2,currencydesc.pickem.votingcoupon.p.pgc2021,open-crate,1
3,currencydesc.pickem.votingcoupon.foc,pickem-pick,-1119226
4,currencydesc.pickem.votingcoupon.p.pickem202206,open-crate,520
5,currencydesc.pickem.votingcoupon.foc,outgameevent-mission-reward,1627938
6,currencydesc.pickem.votingcoupon.p.pickem202206,buy-cash-item,157565
7,currencydesc.pickem.votingcoupon.foc,commercial-word-redeem,26994
8,currencydesc.pickem.votingcoupon.foc,promotion-code-redeem,377


In [82]:
pickem

,Currency,Reason,sum(currency_increased)
0,currencydesc.pickem.votingcoupon.p.pickem202102,open-crate,1
1,currencydesc.pickem.votingcoupon.p,pickem-pick,-116307
2,currencydesc.pickem.votingcoupon.p.pgc2021,open-crate,1
3,currencydesc.pickem.votingcoupon.foc,pickem-pick,-1119226
4,currencydesc.pickem.votingcoupon.p.pickem202206,open-crate,520
5,currencydesc.pickem.votingcoupon.foc,outgameevent-mission-reward,1627938
6,currencydesc.pickem.votingcoupon.p.pickem202206,buy-cash-item,157565
7,currencydesc.pickem.votingcoupon.foc,commercial-word-redeem,26994
8,currencydesc.pickem.votingcoupon.foc,promotion-code-redeem,377
9,currencydesc.pickem.votingcoupon.p.pickem202101,open-crate,5
